In [22]:
import numpy as np
import pandas as pd


In [23]:
runs = !ls *csv
numRep = 4
duration = 8

In [32]:
for run_name in runs:
    subj = run_name.split('_')[1]
    run_num = run_name.split("run",1)[1][0]
    #these are assuming we know the format of file names 
    run = pd.read_csv(run_name)
    #stripping the condition name and creating the sorted list, here also define numRep and duration
    condition_list = {x.lower().replace('fmri_obj/', '').replace('.png', '') for x in run.Object}
    condition_list = pd.DataFrame(condition_list)
    #this can be saved
    #to sort the runs based on the condition, in a similar way the condition_list is created 
    condition_list = condition_list.sort_values(by = condition_list.columns[0]).reset_index(drop=True)[0]
    run.sort_values(by=['Object'], inplace=True)
    if len(run) != numRep*len(condition_list):
        raise Exception('are you sure about the #conditions and repetitions? Check again because this is what my code depends on')
    #split the run log for each condition
    split = [x for _, x in sorted(run.groupby('Object'))]
    for i in range(len(condition_list)):
        ev = np.column_stack([np.sort(split[i].trial_start), np.repeat(duration, numRep) , np.repeat(1, numRep)]) 
        np.savetxt('{}_run-{}.txt'.format(condition_list[i], run_num), ev, fmt='%5s', delimiter=' ')